# Very Simple ODE Model to Test Parameter Optimization

In [2]:
import numpy as np
import scipy.integrate as sci
import matplotlib.pyplot as plt
import mpld3
from scipy.interpolate import interp1d
import scipy.optimize as sco
from tabulate import tabulate
from decimal import Decimal

## Parameters and Initial Conditions

In [5]:
# Initial condition array

y0 = [0]

In [6]:
# optimal parameter values to get the parabola that I'm going to sample data points from
alpha = 1
beta = 0
delta = 0

In [7]:
# bounds for the parameters

bounds = ([0.,5.], [0.,5.])

In [14]:
# time steps to use in integration

timeLength = (0,10)

t_start = -0.01
t_end = 10
t_step = 0.01

## Pull the Data Points Sampled from Desired Parabola from File into Array

In [13]:
rawData = np.genfromtxt('parabola-points.txt')

In [4]:
print(rawData)

[[0.00000000e+00 1.00000000e+00]
 [5.00000000e-01 1.64872127e+00]
 [1.00000000e+00 2.71828183e+00]
 [1.50000000e+00 4.48168907e+00]
 [2.00000000e+00 7.38905610e+00]
 [2.50000000e+00 1.21824940e+01]
 [3.00000000e+00 2.00855369e+01]
 [3.50000000e+00 3.31154520e+01]
 [4.00000000e+00 5.45981500e+01]
 [4.50000000e+00 9.00171313e+01]
 [5.00000000e+00 1.48413159e+02]
 [5.50000000e+00 2.44691932e+02]
 [6.00000000e+00 4.03428793e+02]
 [6.50000000e+00 6.65141633e+02]
 [7.00000000e+00 1.09663316e+03]
 [7.50000000e+00 1.80804241e+03]
 [8.00000000e+00 2.98095799e+03]
 [8.50000000e+00 4.91476884e+03]
 [9.00000000e+00 8.10308393e+03]
 [9.50000000e+00 1.33597268e+04]
 [1.00000000e+01 2.20264658e+04]]


In [3]:
rawData = np.genfromtxt('exp-points.txt')

## Model Function -- Includes ODE Solver

In [9]:
def model(params, ics, t):
    
    # function that constitutes the ODE model
    def ode_system(t, y):
        
        dy = np.zeros(1)
        [alpha, beta] = params
        
        dy[0] = alpha*y[0] + beta
        
        return dy
    
    # solve the system with scipy.integrate.ode to see if it's any faster
    solver = sci.ode(ode_system)
    solver.set_integrator('lsoda')
    solver.set_initial_value(ics, t_start)
    
    ts = []
    ys = []
    
    while solver.successful() and solver.t < t_end:
        solver.integrate(solver.t + t_step)
        ts.append(solver.t)
        ys.append(solver.y)
    
    # reshape the output frome ode to an array with the times on the first column
    ts = np.reshape(ts, (len(ts),1))
    ys = np.vstack(ys)
    timeSeries = np.hstack((ts, ys))
    return timeSeries

## Cost Function

In [24]:
def cost_fun(params):
    simData = model(params, y0, timeLength)
    
    spline = interp1d(simData[:,0], simData[:,1], kind = 'quadratic')
    
    cost = np.sum((spline(raw[0]) - raw[1])**2)
    
    return cost

## Run the Optimization

In [25]:
# number of times to run the optimization
n = 1

# define an array to hold the population of parameter vectors
opt_pars = np.zeros((n, len(bounds)+1))

# initialize array to save simulation data points
sims = np.zeros((1002,n))

In [26]:
%%time

# loop n times, running the optimization each time
for i in range(0,n):
    
    print(f"Optimization Run #{i+1}")
    
    # call the differential evolution optimization function on the cost function
    res = sco.differential_evolution(cost_fun, bounds, maxiter = None, disp = True, popsize = 15)
    
    # alternatively, we can run the SHGO algorithm with the sampling_method = "sobol" flag to do global
    #     optimization with reporting all local minima, as well
    #res = sco.shgo(cost_fun, bounds, callback=callback_fun(*shgo_iter_steps), options = {"f_min": 0.1, "maxiter": None, "minimize_every_iter": True, "local_iter": False, "disp": True}, iters = 3)
    #res = sco.basinhopping(cost_fun, x0, niter = 1000)
    #res = sco.dual_annealing(cost_fun, bounds)
    
    # plug the optimized parameters into the solver
    optimizedSimData = model(res.x, y0, timeLength)
    print(optimizedSimData.shape)
    
    # save CRH, cortisol and ACTH data into sims arrays
    sims[:,i] = optimizedSimData[:,1]
    
    # save the cost function values and optimized parameters for each iteration into the array opt_pars
    opt_pars[i,0] = res.fun
    opt_pars[i,1:] = res.x

Optimization Run #1
differential_evolution step 1: f(x)= 1.58217e-06
differential_evolution step 2: f(x)= 9.1084e-07
differential_evolution step 3: f(x)= 4.05321e-07
differential_evolution step 4: f(x)= 1.34295e-07
differential_evolution step 5: f(x)= 1.34295e-07
differential_evolution step 6: f(x)= 1.02348e-07
differential_evolution step 7: f(x)= 6.53912e-09
differential_evolution step 8: f(x)= 6.53912e-09
differential_evolution step 9: f(x)= 4.37583e-10
differential_evolution step 10: f(x)= 2.2248e-10
differential_evolution step 11: f(x)= 2.2248e-10
differential_evolution step 12: f(x)= 2.2248e-10
differential_evolution step 13: f(x)= 2.2248e-10
differential_evolution step 14: f(x)= 1.06431e-10
differential_evolution step 15: f(x)= 3.04851e-12
differential_evolution step 16: f(x)= 3.04851e-12
differential_evolution step 17: f(x)= 3.04851e-12
differential_evolution step 18: f(x)= 1.98518e-13
differential_evolution step 19: f(x)= 1.35427e-13
differential_evolution step 20: f(x)= 1.3542

differential_evolution step 164: f(x)= 1.11343e-34
differential_evolution step 165: f(x)= 1.11343e-34
differential_evolution step 166: f(x)= 1.11343e-34
differential_evolution step 167: f(x)= 1.11343e-34
differential_evolution step 168: f(x)= 1.11343e-34
differential_evolution step 169: f(x)= 1.11343e-34
differential_evolution step 170: f(x)= 2.40741e-35
differential_evolution step 171: f(x)= 2.40741e-35
differential_evolution step 172: f(x)= 2.40741e-35
differential_evolution step 173: f(x)= 2.40741e-35
differential_evolution step 174: f(x)= 2.40741e-35
differential_evolution step 175: f(x)= 2.40741e-35
differential_evolution step 176: f(x)= 2.40741e-35
differential_evolution step 177: f(x)= 2.40741e-35
differential_evolution step 178: f(x)= 2.40741e-35
differential_evolution step 179: f(x)= 2.40741e-35
differential_evolution step 180: f(x)= 2.40741e-35
(1002, 2)
CPU times: user 53.1 s, sys: 4.03 s, total: 57.1 s
Wall time: 56.6 s


In [27]:
print(opt_pars)

[[2.40741243e-35 1.48922757e+00 9.92572344e-01]]


In [22]:
raw = model([1,1], y0, timeLength)

In [30]:
print(model([1.48922757,9.92572344e-01], y0, timeLength))

[[0.00000000e+00 1.00000000e-02]
 [1.00000000e-02 2.01500372e-02]
 [2.00000000e-02 3.04523627e-02]
 ...
 [9.99000000e+00 1.95631054e+06]
 [1.00000000e+01 1.98566252e+06]
 [1.00100000e+01 2.01545488e+06]]


In [32]:
print(raw-model([1.48922757,9.92572344e-01], y0, timeLength))

[[ 0.00000000e+00  5.01670812e-05]
 [ 0.00000000e+00  5.13028390e-05]
 [ 0.00000000e+00  2.17128466e-06]
 ...
 [ 0.00000000e+00 -1.93428506e+06]
 [ 0.00000000e+00 -1.96341567e+06]
 [ 0.00000000e+00 -1.99298443e+06]]


In [33]:
print(raw)

[[0.00000000e+00 1.00501671e-02]
 [1.00000000e-02 2.02013400e-02]
 [2.00000000e-02 3.04545340e-02]
 ...
 [9.99000000e+00 2.20254805e+04]
 [1.00000000e+01 2.22468503e+04]
 [1.00100000e+01 2.24704450e+04]]
